In [1]:
import pandas as pd
import re


In [108]:
def preprocess(data):
    datp1 = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'
    datp2 = '^(\d{1,2}\/\d{1,2}\/\d{1,2}, \d{1,2}:\d{1,2} \w\w)'
    datp3 = "\[\d{1,2}/\d{1,2}/\d{2,4}, \d{2}:\d{2}:\d{2} [APMapm]{2}\] \w+: .*"

    pattern =  datp1 or  datp2 or datp3
    messages = re.split(pattern, data)[1:]
    dates = re.findall(pattern, data)
    
    df = pd.DataFrame({'user_message': messages, 'message_date': dates})
    
    # Converting Message Date Type...
    df['message_date'] = pd.to_datetime(df['message_date'], format='%d/%m/%Y, %H:%M - ', errors='coerce')
    df.rename(columns={'message_date': 'date'}, inplace=True)
    
    # Separate users & Messages
    users = []
    messages = []
    
    for message in df['user_message']:
        entry = re.split('([\w\W]+?):\s', message)
        if entry[1:]:  # user name
            users.append(entry[1])
            messages.append(" ".join(entry[2:]))
        else:
            users.append('group_notification')
            messages.append(entry[0])

    df['user'] = users
    df['message'] = messages
    df.drop(columns=['user_message'], inplace=True)
    
    df['year'] = df['date'].dt.year
    df['month_num'] = df['date'].dt.month
    df['month'] = df['date'].dt.month_name()
    df['day'] = df['date'].dt.day
    df['day_name'] = df['date'].dt.day_name()

    df['hour'] = df['date'].dt.hour
    df['minute'] = df['date'].dt.minute

    df['only_date'] = df['date'].dt.date

    return df

In [140]:
chat_location = "dateF1.txt"

In [141]:
with open(chat_location, 'rb') as file:
        bytes_data = file.read()
        data = bytes_data.decode("utf-8")
    # bytes_data = chat_location.getvalue()
    # data = bytes_data.decode("utf-8") 
# print("Raw Data:", data)

In [161]:
chat_location2 = "ios.txt"
with open(chat_location2, 'rb') as file:
        bytes_data = file.read()
        data2 = bytes_data.decode("utf-8")

In [169]:
import pandas as pd

def preprocess(data):
    messages = []
    dates = []
    is_ios = False

    for line in data.split('\n'):
        if line.strip() == "":
            continue

        parts = line.split(" - ", 1)
        if len(parts) > 1:
            date_str, message = parts
            dates.append(pd.to_datetime(date_str.strip(), errors='coerce'))
            messages.append(message)
        elif line.startswith("["):
            is_ios = True
            date_str, message = line.split("] ", 1)
            dates.append(pd.to_datetime(date_str[1:].strip(), format='%d/%m/%y, %I:%M:%S %p', errors='coerce'))
            messages.append(message)

        # elif line.startswith("["):
        #     is_ios = True
        #     date_str, message = line.split("] ", 1)
        #     dates.append(pd.to_datetime(date_str[1:].strip(), format='%d/%m/%y, %I:%M:%S%p', errors='coerce'))
        #     messages.append(message)
        else:
            # Handle the case where a message starts without a timestamp (continuation of the previous message)
            messages[-1] += " " + line

    df = pd.DataFrame({'user_message': messages, 'message_date': dates})

    # Converting Message Date Type...
    df['message_date'] = pd.to_datetime(df['message_date'], errors='coerce')
    df.rename(columns={'message_date': 'date'}, inplace=True)

    # Separate users & Messages
    users = []
    messages = []

    for message in df['user_message']:
        entry = message.split(': ', 1)
        if len(entry) > 1:
            users.append(entry[0])
            messages.append(entry[1])
        else:
            users.append('group_notification')
            messages.append(entry[0])

    df['user'] = users
    df['message'] = messages
    df.drop(columns=['user_message'], inplace=True)

    df['year'] = df['date'].dt.year
    df['month_num'] = df['date'].dt.month
    df['month'] = df['date'].dt.month_name()
    df['day'] = df['date'].dt.day
    df['day_name'] = df['date'].dt.day_name()

    df['hour'] = df['date'].dt.hour
    df['minute'] = df['date'].dt.minute

    df['only_date'] = df['date'].dt.date

    # Add braces to text in a for loop
   # Add braces to text in a for loop
    for i in range(len(df)):
        df.loc[i, 'message'] = f"{{{df.loc[i, 'message']}}}"

    # Remove braces from text in a for loop
    for i in range(len(df)):
        df.loc[i, 'message'] = df.loc[i, 'message'].strip("{}")


    return df, is_ios



In [159]:
df,df_ios=preprocess(data)
df.sample(5)

,date,user,message,year,month_num,month,day,day_name,hour,minute,only_date
22,2023-12-19 21:45:00,A N K I T,Already bht applications banaye hai & even as ...,2023,12,December,19,Tuesday,21,45,2023-12-19
5,2022-10-18 19:57:00,A N K I T,Web development,2022,10,October,18,Tuesday,19,57,2022-10-18
6,2022-11-16 01:39:00,group_notification,Your security code with Bollywood Chaiwala cha...,2022,11,November,16,Wednesday,1,39,2022-11-16
10,2023-09-13 12:15:00,group_notification,Your security code with Bollywood Chaiwala cha...,2023,9,September,13,Wednesday,12,15,2023-09-13
18,2023-12-19 19:25:00,A N K I T,Mobile application,2023,12,December,19,Tuesday,19,25,2023-12-19


In [170]:
df,df_ios=preprocess(data2)
df.sample(5)

,date,user,message,year,month_num,month,day,day_name,hour,minute,only_date
629,2023-11-26 00:36:36,Vishal Gupta,Ha dekho\r,2023,11,November,26,Sunday,0,36,2023-11-26
321,2023-11-17 11:04:50,Kapish,Acha\r,2023,11,November,17,Friday,11,4,2023-11-17
668,2023-11-28 22:02:49,Ankit Scl,areah kaise hua\r,2023,11,November,28,Tuesday,22,2,2023-11-28
853,2023-12-01 20:53:09,Vishal Gupta,"Ok\r ‎[01/12/23, 9:44:27 PM] Ankit Scl: ‎<atta...",2023,12,December,1,Friday,20,53,2023-12-01
386,2023-11-17 17:01:44,Vishal Gupta,"Ok😂\r ‎[17/11/23, 5:10:14 PM] Vishal Gupta: ‎<...",2023,11,November,17,Friday,17,1,2023-11-17
